## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import matplotlib.pyplot as plt
import scipy.signal as sg
%matplotlib qt

In [2]:
import glob

import numpy as np
import scipy as sp
import scipy.io


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)

    # Compute pulse rate estimates and estimation confidence.

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    errors, confidence = np.ones(100), np.ones(100)  # Dummy placeholders. Remove
    return errors, confidence

In [4]:
def Load_labels(data_fl,ts):
    """
    Loads and extracts signals from a troika data file.
    Perform interpolation for converting the data to the TS of the measurements
    Args:
        data_fl: (str) filepath to a troika .mat file.
        ts: time stamp of the sensors measurements
    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    # as mentioed in the dataset read_me the ref value is obtained using a window of 8s and 6s overlap
    # so we can assume that the sample freq of the ref data is 2s
    fs_labels=2
    data = sp.io.loadmat(data_fl)['BPM0']
    ts_labels=np.arange(0,len(data)/fs_labels,1/fs_labels)
    labels_interp=np.interp(ts,ts_labels,data[:,0])
    return labels_interp

In [5]:

def butter_bandpass(data,rang,fs):
    results={}
    zeros,poles=sg.butter(5,rang,btype="bandpass",fs=fs)
    for key in data :
        results[key]=sg.filtfilt(zeros,poles,data[key])
    return results
    

In [6]:
def fft_sensors(data,fs):
    ff_data={}
    for key in data :
        ff_freq=np.fft.rfftfreq(len(data[key]),1./fs)
        ff_power=np.fft.rfft(data[key])
        ff_data[key]=np.array([ff_freq,ff_power])
    return ff_data
def fft_inv(fftdata):
    data={}
    for key in fftdata :
        data[key]=np.fft.irfft(fftdata[key][1])
    return data

In [7]:
def vizualize_spectugram(data_dict_td,ts,wind_title,ylim=(0,6),nfft=1000,overlap=875):
    data={}
    fs=125
    len_dict=len(data_dict_td)
    plt.clf
    fig,ax=plt.subplots(len_dict,1,figsize=(10,6*len_dict))
    fig.canvas.set_window_title(wind_title)
    ax=ax.flatten()
    for key,axes in zip(data_dict_td,ax):
        tss=np.arange(0,len(data_dict_td[key]),1/fs)
        spec,freqs,_,_=axes.specgram(data_dict_td[key],Fs=fs,NFFT=nfft,noverlap=overlap, xextent=(0, tss[-1]),pad_to=1024)
        data[key]=(spec,freqs)
        axes.set_ylim(ylim)
        axes.set_ylabel('%s Frequency (Hz)'%key)
        #axes.set_xlabel('Time (sec)')
    return data

In [8]:
def vizualize_data(data_dict_fs,wind_title,xlim,type="freq"):
    len_dict=len(data_dict_fs)
    plt.clf
    fig,ax=plt.subplots(len_dict,1,figsize=(10,6*len_dict))
    fig.canvas.set_window_title(wind_title)
    ax=ax.flatten()
    for key,axes in zip(data_dict_fs,ax):
        if type=="freq":
            power=np.abs(data_dict_fs[key][1])
            freqs=np.abs(data_dict_fs[key][0])
            axes.plot(freqs,power,label=key)
            axes.set_xlabel(key)
            axes.set_xlim(xlim)
            axes.legend()
        else:
            axes.plot(data_dict_fs[key],label=key)
            axes.set_xlabel(key)
            axes.set_xlim(xlim)
            axes.legend()
            
        #axes.set_xlabel('Time (sec)')

In [9]:
import copy

In [ ]:
def load_all_dataset(data_fls,ref_fls):
    '''
    Load data, labels from all files, split, shuffle
    return samples and their labels
    each sample is a Xs window with overlab of Xs 
    '''
    return x,y

In [49]:
##Reading ref and sensors data, create timestamp for both

data_fls, ref_fls = LoadTroikaDataset()

ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])
fs=125.
ts=np.arange(0,len(ppg)/fs,1/fs)
labels=Load_labels(ref_fls[0],ts)
len(ppg),len(accx),len(accy),len(accz),len(labels)

(37937, 37937, 37937, 37937, 37937)

In [63]:
res_tes=[np.mean(labels[i*125:i*125+125])/60. for i in range(int(len(labels)/125.))]

In [64]:
res_tes

[1.267300245887308,
 1.2463866902654865,
 1.2034962818458392,
 1.2295367817768734,
 1.285642727343025,
 1.301714516075459,
 1.1910260453301298,
 1.211119127790551,
 1.3362633668324613,
 1.3938854127308504,
 1.4665787777590222,
 1.5474754834943902,
 1.6204215918471039,
 1.690105082318241,
 1.720474900865822,
 1.7182091600594207,
 1.739822189918643,
 1.8082247961179223,
 1.8587913064780035,
 1.8811598836663845,
 1.907298489109658,
 1.9341743284373651,
 1.9762111757609078,
 2.032788311560399,
 2.0980915839141443,
 2.178728165575082,
 2.2554357775492937,
 2.3145506942390264,
 2.3606715295150775,
 2.383439527876827,
 2.3916814710953096,
 2.404299308473547,
 2.42959129318736,
 2.4719525652202345,
 2.5198142234664265,
 2.560565926669031,
 2.585873176956391,
 2.587357483164811,
 2.5790279351538037,
 2.5829293092889665,
 2.584658353773705,
 2.5721045284972246,
 2.5593494138569666,
 2.5437720742647167,
 2.5278547749298266,
 2.5275798822961733,
 2.533333967454995,
 2.5293094581473707,
 2.52591528

Pack all data into single dictionary

In [11]:
#create dictionary for all the data , filter the data with bandpass in 0.67-6Hz
sensors_data={"ppg":ppg,"accx":accx,"accy":accy,"accz":accz}

Filter frequencies between 40-240 PBM

In [12]:

data_filtered=butter_bandpass(sensors_data,(0.6,4),fs)

In [13]:
##convert to freq domain and visualize 
fft_data=fft_sensors(data_filtered,fs)
##get the most powerful frequencies in the acceleromenter
fft_acc_sum=fft_data['accx'][1]+fft_data['accy'][1]+fft_data['accz'][1]
fft_acc_sum/=3.
#scale down most affeted frequencies by acc
fft_filtered_data=copy.deepcopy(fft_data)
fft_filtered_data['ppg'][1][(-fft_acc_sum).argsort()[:200]]*=0.1
#back to time domain
filtered_from_acc_data=fft_inv(fft_filtered_data)

In [14]:
#do visualization here

#Spectugram visualizations
spec_raw=vizualize_spectugram(sensors_data,ts,
                     wind_title="raw_data",
                     ylim=(0,6))
spec_filtered=vizualize_spectugram(data_filtered,ts,
                     wind_title="0.5-6.0Hz filtered",
                     ylim=(0,6))
spec_acc_freqs_removed=vizualize_spectugram(filtered_from_acc_data,ts,
                     wind_title="powerful_acc_freqs_filtered",
                     ylim=(0,6))


In [16]:
spec_filtered_acc=copy.deepcopy(spec_filtered)
accel_ccc_spec=spec_filtered['accx'][0]+spec_filtered['accy'][0]+spec_filtered['accz'][0]
acc_powerful_freqs_in_each_window=[]
n_freq=4
for ind in range(accel_ccc_spec.shape[1]):
    window= accel_ccc_spec[:,ind]
    window_strong_freqs_indexes=(-window).argsort()[:n_freq]
    spec_filtered_acc['ppg'][0][:,ind][window_strong_freqs_indexes]*=0.1
    
    

In [17]:
spec_filtered['ppg'][1].shape

(513,)

In [18]:
pred=spec_filtered['ppg'][1][accel_ccc_spec.argmax(axis=0)]


In [20]:
pred

array([0.61035156, 0.61035156, 0.73242188, 0.73242188, 0.73242188,
       0.73242188, 0.9765625 , 0.9765625 , 0.9765625 , 0.9765625 ,
       0.85449219, 0.73242188, 0.73242188, 0.73242188, 0.85449219,
       0.85449219, 0.85449219, 1.83105469, 1.83105469, 1.34277344,
       1.34277344, 1.22070312, 1.22070312, 0.9765625 , 1.09863281,
       1.09863281, 0.9765625 , 0.9765625 , 0.9765625 , 1.22070312,
       1.22070312, 1.22070312, 0.73242188, 0.73242188, 0.73242188,
       0.73242188, 0.85449219, 1.953125  , 1.953125  , 1.953125  ,
       1.953125  , 2.68554688, 2.68554688, 2.68554688, 2.68554688,
       2.68554688, 2.68554688, 2.68554688, 2.56347656, 2.56347656,
       2.56347656, 2.56347656, 2.56347656, 2.68554688, 2.68554688,
       2.68554688, 2.68554688, 2.56347656, 2.56347656, 2.56347656,
       2.56347656, 2.56347656, 2.56347656, 2.56347656, 2.68554688,
       2.68554688, 2.56347656, 2.56347656, 2.56347656, 2.56347656,
       2.56347656, 2.56347656, 2.56347656, 2.68554688, 2.68554

In [21]:
pred.mean()

2.4178994668496623

In [22]:
pred[-50:]

array([2.9296875 , 2.9296875 , 2.9296875 , 2.9296875 , 2.9296875 ,
       2.9296875 , 2.9296875 , 2.9296875 , 2.9296875 , 2.9296875 ,
       2.9296875 , 2.9296875 , 2.9296875 , 2.9296875 , 2.9296875 ,
       2.9296875 , 2.9296875 , 2.9296875 , 2.9296875 , 2.9296875 ,
       2.9296875 , 2.9296875 , 2.80761719, 2.80761719, 2.80761719,
       2.9296875 , 2.9296875 , 3.05175781, 2.80761719, 1.46484375,
       0.85449219, 0.85449219, 0.73242188, 0.73242188, 0.73242188,
       0.73242188, 0.73242188, 0.73242188, 0.73242188, 0.73242188,
       1.46484375, 1.46484375, 1.46484375, 1.70898438, 1.953125  ,
       1.83105469, 2.68554688, 2.68554688, 1.34277344, 0.73242188])

In [23]:
labels/=60.0


ended here  create rolling window over labels to conver to 1 s measurement

In [ ]:
labels.mean()

In [ ]:
plt.plot(spec_filtered['ppg'][1],spec_filtered['ppg'][0][:,0])

In [ ]:
#Freq visualizations
#%matplotlib inline
vizualize_data(fft_data,
               wind_title="Freq Domain",
               xlim=(0,6)
              )
vizualize_data(fft_filtered_data,
               wind_title="PPG filtered from powerful acc freqs",
               xlim=(0,6)
              )

In [ ]:
vizualize_data(data_filtered,
               wind_title="time filtered Domain",
              xlim=(0,10000),
               type="time"
              )

In [ ]:
vizualize_data(filtered_from_acc_data,
               wind_title="time acc removed Domain",
              xlim=(0,10000),
               type="time"
              )

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

-----
### Next Steps
You will now go to **Test Your Algorithm** (back in the Project Classroom) to apply a unit test to confirm that your algorithm met the success criteria. 